In [14]:
# IMPORT THE LIBRARIES
import pandas as pd
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import json
import pprint

import os
import re
import requests
import time

In [2]:
# Spotify API KEYS
client_id = 'f9d64942f5f74f3282916225d2f77630'
client_secret = 'cc318334877a40cfae2d59e07ae86d9f'
client_credentials_manager = SpotifyClientCredentials('f9d64942f5f74f3282916225d2f77630', 'cc318334877a40cfae2d59e07ae86d9f')
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [3]:
guitar = pd.read_csv("https://github.com/TStrada/Dataset/raw/main/Dataset/Dataset_Song_Features_clean.csv", sep = ",")
guitar['Song'] = guitar['Song'].astype(str).apply(lambda x: re.sub(r"\((.*?)\)", "", x))
guitar.head(30)

,Unnamed: 0,Band,Song,Player,Technique,Difficulty,Score
0,0,The Beatles,Blackbird,Paul McCartney,"['Guitar', 'Acoustic Guitar (nylon)']",Intermediate,3/8
1,1,Isaac Albéniz,Asturias,Ýz 1,['Acoustic Guitar (nylon)'],Intermediate,5/8
2,2,AC/DC,Back In Black,Angus Young (Lead),['Distortion Guitar'],Intermediate,3/8
3,3,Pachelbel,Canon in D Major,NaN,['Acoustic Guitar (nylon)'],Intermediate,4/8
4,4,Led Zeppelin,"Babe, I'm Gonna Leave You",Jimmy Page,['Acoustic Guitar (steel)'],Beginner,2/8
5,5,Jimi Hendrix,Little Wing,Jimi Hendrix,['Electric Guitar (jazz) 1'],Intermediate,5/8
6,6,Pink Floyd,Shine On You Crazy Diamond,David Gilmour,"['Lap Steel Harmony', 'Overdriven Guitar']",Beginner,2/8
7,7,Kansas,Dust In The Wind,NaN,"['Acoustic Guitar (nylon)', 'Guitar 1']",Intermediate,3/8
8,8,Francisco Tarrega,Capricho Arabe,NaN,"['Acoustic Guitar (nylon)', 'Serenata para Gui...",Intermediate,5/8
9,9,Pachelbel,Canon In C,NaN,['Acoustic Guitar (nylon)'],Beginner,1/8


In [4]:
df = guitar #copy
df.head()

,Unnamed: 0,Band,Song,Player,Technique,Difficulty,Score
0,0,The Beatles,Blackbird,Paul McCartney,"['Guitar', 'Acoustic Guitar (nylon)']",Intermediate,3/8
1,1,Isaac Albéniz,Asturias,Ýz 1,['Acoustic Guitar (nylon)'],Intermediate,5/8
2,2,AC/DC,Back In Black,Angus Young (Lead),['Distortion Guitar'],Intermediate,3/8
3,3,Pachelbel,Canon in D Major,NaN,['Acoustic Guitar (nylon)'],Intermediate,4/8
4,4,Led Zeppelin,"Babe, I'm Gonna Leave You",Jimmy Page,['Acoustic Guitar (steel)'],Beginner,2/8


In [5]:
# Define features columns

df["ID_TRACK"] = ""
df["POPULARITY"] = ""
df["RELEASE_DATE"] = ""
df["EXTERNAL_URLS"] = ""
df["SEARCH_URL"] = ""
# df["AVAILABLE_MARKETS"] = ""       # NON CI INTERESSA ?    
df["danceability"] = ""
df["energy"] = ""
df["loudness"] = ""
df["speechiness"] = ""
df["acousticness"] = ""
df["instrumentalness"] = ""
df["liveness"] = ""
df["valence"] = ""
df["tempo"] = ""
df["duration_ms"] = ""
#df["key"] = ""
#df["mode"] = ""
#df["beats"] = ""

In [6]:
def find_track_ID(artist, track,i):

        track_id = sp.search(q='artist:' + artist + ' track:' + track, type='track')    
        #pprint.pprint(track_id)
        ID_TRACK = track_id['tracks']['items'][0]['id']
        #print(ID_TRACK)
        POPULARITY = track_id['tracks']['items'][0]['popularity']
        #print(POPULARITY)
        RELEASE_DATE = track_id['tracks']['items'][0]['album']['release_date']
        EXTERNAL_URLS = track_id['tracks']['items'][0]['album']['external_urls']['spotify']
        SEARCH_URL = track_id['tracks']['items'][0]['preview_url']
#        AVAILABLE_MARKETS = len(track_id['tracks']['items'][0]['album']['available_markets'])

        
        df["ID_TRACK"][i] = ID_TRACK
        df["POPULARITY"][i] = POPULARITY
        df["RELEASE_DATE"][i] = RELEASE_DATE
        df["EXTERNAL_URLS"][i] = EXTERNAL_URLS
        df["SEARCH_URL"][i] = SEARCH_URL
#        df["AVAILABLE_MARKETS"][i] = AVAILABLE_MARKETS     

        #audio_features
        af = sp.audio_features(str(ID_TRACK))[0]
        
        df["danceability"][i]= af["danceability"]
        df["energy"][i] = af["energy"]
        df["loudness"][i] = af["loudness"]
        df["speechiness"][i] = af["speechiness"]
        df["acousticness"][i] = af["acousticness"]
        df["instrumentalness"][i] = af["instrumentalness"]
        df["liveness"][i] = af["liveness"]
        df["valence"][i] = af["valence"]
        df["tempo"][i] = af["tempo"]
        df["duration_ms"][i] = af["duration_ms"]
    
        #audio_analysis                                   # NON FUNZIONA
#        aa = sp.audio_analysis(str(ID_TRACK))[0]

#        df["key"][i] = aa["key"]
#        df["mode"][i] = aa["mode"]
#       df["beats"][i] = aa["beats"]



In [7]:
error = []
found = []
for i in range(len(df)): 
    try:
        find_track_ID(df["Band"].iloc[i],df["Song"].iloc[i], i)
        print(i)
        found.append(i)
    except:
        print("Error: element not found")
        df["ID_TRACK"][i] = None
        df["POPULARITY"][i] = None
        df["RELEASE_DATE"][i] = None
        df["EXTERNAL_URLS"][i] = None
        df["SEARCH_URL"][i] = None
#        df["AVAILABLE_MARKETS"][i] = None
        df["danceability"][i]= None
        df["energy"][i] = None
        df["loudness"][i] = None
        df["speechiness"][i] = None
        df["acousticness"][i] = None
        df["instrumentalness"][i] = None
        df["liveness"][i] = None
        df["valence"][i] = None
        df["tempo"][i] = None
        df["duration_ms"][i] = None
        error.append(i)
    
    
print("")
print(f'{len(found)} elements found, {len(error)} elements not found')

C:\Users\campa\AppData\Local\Temp\ipykernel_4524\433667329.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["ID_TRACK"][i] = ID_TRACK
C:\Users\campa\AppData\Local\Temp\ipykernel_4524\433667329.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["POPULARITY"][i] = POPULARITY
C:\Users\campa\AppData\Local\Temp\ipykernel_4524\433667329.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["RELEASE_DATE"][i] = RELEASE_DATE
C:\U

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Error: element not found


C:\Users\campa\AppData\Local\Temp\ipykernel_4524\1372380371.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["ID_TRACK"][i] = None
C:\Users\campa\AppData\Local\Temp\ipykernel_4524\1372380371.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["POPULARITY"][i] = None
C:\Users\campa\AppData\Local\Temp\ipykernel_4524\1372380371.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["RELEASE_DATE"][i] = None
C:\Users\campa\AppD

16
17
18
19
20
21
22
Error: element not found
24
Error: element not found
26
27
28
Error: element not found
30
31
32
33
34
Error: element not found
36
37
Error: element not found
39
40
41
42
43
44
45
Error: element not found
47
48
49
50
51
52
53
54
55
56
57
Error: element not found
59
60
61
62
63
64
Error: element not found
Error: element not found
67
68
Error: element not found
70
71
Error: element not found
73
Error: element not found
Error: element not found
76
77
78
79
Error: element not found
81
Error: element not found
Error: element not found
Error: element not found
85
86
87
88
89
90
91
92
Error: element not found
Error: element not found
95
96
97
98
99
Error: element not found
101
102
103
104
105
106
107
108
Error: element not found
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
Error: element not found
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
Error: element not found
150
151
152
153
154
155
156
157
158
159
160
16

Error: element not found
791
Error: element not found
Error: element not found
794
795
796
Error: element not found
Error: element not found
Error: element not found
Error: element not found
801
Error: element not found
803
804
Error: element not found
Error: element not found
Error: element not found
Error: element not found
Error: element not found
Error: element not found
811
Error: element not found
Error: element not found
814
Error: element not found
816
817
818
Error: element not found
820
Error: element not found
Error: element not found
Error: element not found
Error: element not found
Error: element not found
Error: element not found
827
828
Error: element not found
Error: element not found
831
Error: element not found
833
Error: element not found
835
Error: element not found
837
Error: element not found
839
Error: element not found
Error: element not found
Error: element not found
Error: element not found
Error: element not found
845
846
Error: element not found
848
849
850


...

Error: element not found
Error: element not found
Error: element not found
Error: element not found
3635
3636
Error: element not found
Error: element not found
Error: element not found
Error: element not found
Error: element not found
Error: element not found
3643
Error: element not found
3645
3646
3647
3648
Error: element not found
Error: element not found
3651
3652
Error: element not found
Error: element not found
Error: element not found
Error: element not found
Error: element not found
Error: element not found
Error: element not found
3660
3661
3662
Error: element not found
Error: element not found
Error: element not found
Error: element not found
Error: element not found
Error: element not found
Error: element not found
Error: element not found
Error: element not found
3672
Error: element not found
3674
Error: element not found
Error: element not found
Error: element not found
Error: element not found
Error: element not found
Error: element not found
Error: element not found
Error

4123
4124
Error: element not found
Error: element not found
4127
Error: element not found
4129
Error: element not found
Error: element not found
4132
Error: element not found
Error: element not found
4135
Error: element not found
Error: element not found
4138
4139
Error: element not found
Error: element not found
Error: element not found
Error: element not found
Error: element not found
4145
Error: element not found
4147
4148
Error: element not found
Error: element not found
4151
4152
4153
Error: element not found
Error: element not found
Error: element not found
Error: element not found
Error: element not found
Error: element not found
4160
Error: element not found
Error: element not found
Error: element not found
Error: element not found
Error: element not found
4166
4167
Error: element not found
Error: element not found
Error: element not found
Error: element not found
Error: element not found
4173
4174
Error: element not found
4176
4177
Error: element not found
Error: element not f

Error: element not found
Error: element not found
Error: element not found
Error: element not found
Error: element not found
Error: element not found
Error: element not found
Error: element not found
Error: element not found
Error: element not found
Error: element not found
Error: element not found
4587
Error: element not found
Error: element not found
Error: element not found
Error: element not found
4592
Error: element not found
4594
Error: element not found
4596
4597
Error: element not found
Error: element not found
4600
Error: element not found
Error: element not found
Error: element not found
Error: element not found
4605
4606
Error: element not found
4608
Error: element not found
Error: element not found
Error: element not found
Error: element not found
4613
Error: element not found
Error: element not found
4616
4617
4618
4619
Error: element not found
4621
4622
Error: element not found
Error: element not found
Error: element not found
4626
Error: element not found
Error: element 

HTTP Error for GET to https://api.spotify.com/v1/search with Params: {'q': 'artist:Continuous Atrocities Against The Condemned Species For The Purpose Of Self-Preservation From The Inevitable Consequences Of Playing God Infront The Sights Of Nature And Existance track:Rage...Pure Unfiltered, Uncontrollable And Unstoppable Rage Spewing From The Blood Vessels, Causing Shaking Within The Body', 'limit': 10, 'offset': 0, 'type': 'track', 'market': None} returned 404 due to Not found.
HTTP Error for GET to https://api.spotify.com/v1/search with Params: {'q': 'artist:Continuous Atrocities Against The Condemned Species For The Purpose Of Self-Preservation From The Inevitable Consequences Of Playing God Infront The Sights Of Nature And Existance track:Witnessing Every Human Act Of Disrespect Towards Their Settlement As The Old Man Barely Breathes With His Lungs, His Cries Remain Unheard Because His Neighbours Never Went Through The Path He Chose', 'limit': 10, 'offset': 0, 'type': 'track', 'ma

Error: element not found
Error: element not found
Error: element not found
Error: element not found
Error: element not found
4761
Error: element not found
Error: element not found
Error: element not found
Error: element not found
4766
Error: element not found
Error: element not found
4769
Error: element not found
4771
Error: element not found
Error: element not found
Error: element not found
Error: element not found
Error: element not found
4777
Error: element not found
Error: element not found
Error: element not found
Error: element not found
4782
Error: element not found
Error: element not found
4785
4786
4787
Error: element not found
4789
Error: element not found
4791
Error: element not found
Error: element not found
Error: element not found
4795
Error: element not found
Error: element not found
4798
Error: element not found
Error: element not found
4801
4802
4803
Error: element not found
Error: element not found
Error: element not found
Error: element not found
Error: element not f

In [8]:
df = df.drop(['Player', 'Technique', 'Difficulty', 'Score'], axis = 1)
df

,Unnamed: 0,Band,Song,ID_TRACK,POPULARITY,RELEASE_DATE,EXTERNAL_URLS,SEARCH_URL,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,0,The Beatles,Blackbird,5jgFfDIR6FR0gvlA56Nakr,72,1968-11-22,https://open.spotify.com/album/1klALx0u4AavZNE...,None,0.686,0.127,-14.361,0.0342,0.754,0.000014,0.0573,0.372,93.699,138387
1,1,Isaac Albéniz,Asturias,2u6GTNZF07O4mmFg8CZozY,36,1981,https://open.spotify.com/album/5lY62FNmxjms4v3...,https://p.scdn.co/mp3-preview/5e9af1c3d16503e3...,0.32,0.171,-20.784,0.0594,0.967,0.884,0.104,0.217,112.092,372733
2,2,AC/DC,Back In Black,08mG3Y1vljYA6bvDt4Wqkj,84,1980-07-25,https://open.spotify.com/album/6mUdeDZCsExyJLM...,https://p.scdn.co/mp3-preview/41f8586f16bc642b...,0.31,0.7,-5.678,0.047,0.011,0.00965,0.0828,0.763,188.386,255493
3,3,Pachelbel,Canon in D Major,1Mse9NKBbEASi50CQ4aYhr,55,1986,https://open.spotify.com/album/5YCuibCDJrkVcS3...,https://p.scdn.co/mp3-preview/eaec965ecde7c7b9...,0.128,0.132,-17.67,0.0353,0.744,0.942,0.107,0.0988,85.314,301587
4,4,Led Zeppelin,"Babe, I'm Gonna Leave You",4OMu5a8sFpcRCPCcsoEaov,61,1969-01-12,https://open.spotify.com/album/3ycjBixZf7S3WpC...,https://p.scdn.co/mp3-preview/e68e42946f936dc7...,0.404,0.435,-12.045,0.0323,0.246,0.00074,0.156,0.156,136.058,401067
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4817,4817,Your Mumies,play.,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4818,4818,Slaver,The moon,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4819,4819,Vundabar,Alien Blues,11iIikXxC6NP0Ma8vMD27x,81,2015-07-24,https://open.spotify.com/album/1vWOYk3hF5bgVUU...,https://p.scdn.co/mp3-preview/55e339f77b534ca0...,0.467,0.757,-5.093,0.033,0.0807,0.908,0.0876,0.433,82.282,155933
4820,4820,Paul McCartney,Uncle Albert/Admiral Halsey,5SwgE4wA0P7f6AEz2UQMB2,54,1971-05-17,https://open.spotify.com/album/3DTMsrNO6lEHNmD...,None,0.371,0.507,-9.805,0.0325,0.58,0.0201,0.175,0.535,91.786,295147


In [9]:
df.to_csv('spotify_features.csv') 

In [10]:
# WAV

# ## Define a function:
# - Input = artist name, song name, url preview
# - Output = Database(columns = 'Artist name', 'Song name', 'song.wav'

def downloader(artist, song, url):
    
    wav_song = artist + ' - ' + song + '.wav'
    
    try:
        sample = requests.get(url)
        open(wav_song, 'wb').write(sample.content)
    except:
        pass
    
    
    return(wav_song)

In [11]:
# MP3

# ## Define a function:
# - Input = artist name, song name, url preview
# - Output = Database(columns = 'Artist name', 'Song name', 'song.mp3'

def downloader_mp3(artist, song, url):
    
    mp3_song = artist + '-' + song + '.mp3'
    
    try:
        sample = requests.get(url)
        open(mp3_song, 'wb').write(sample.content)
    except:
        pass
    
    
    return(mp3_song)

In [12]:
df['Song'][[0]]

0    Blackbird
Name: Song, dtype: object

In [16]:
start = time.perf_counter()
for track in range(len(df)):
    downloader_mp3(df['Band'][track], df['Song'][track], df['SEARCH_URL'][track])
print(f'seconds elapsed: {time.perf_counter()-start}')

466.4731503999999
